In [2]:
import re
import nltk
import faker
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
zoo_v6_chat_id = -1001080678305

In [4]:
messages = pd.read_json("datasets/translated_messages.json")

In [5]:
len(messages)

1000

In [7]:
# remove messages with no text
messages = messages[messages.text.notnull()]
# remove messages where text starts with '/'
messages = messages[~messages["text"].str.startswith("/")]
# remove messages via bot
messages = messages[~messages["via_bot"].notnull()]
# drop columns
messages.drop(['audio', 'media_group_id', 'video_note', 'voice', 'poll', 'video', 'web_page', 'reply_markup', 'caption_entities', 'service',
               'via_bot', 'has_media_spoiler', 'photo', 'sticker', 'animation', 'document', 'scheduled', 'from_scheduled', 'has_protected_content',
               'entities', 'media', 'new_chat_members', 'forward_from', 'forwarded_from_chat', 'caption'],
              inplace=True, axis=1, errors='ignore')

In [8]:
def reactions_to_count(x):
    return sum(r["count"] for r in x["reactions"]) if x is not None and x is not np.nan else 0


messages["reactions_count"] = messages["reactions"].apply(reactions_to_count)

In [9]:
messages["author_id"] = messages["from_user"].apply(lambda x: x["id"])
messages["author_username"] = messages["from_user"].apply(lambda x: x["username"])

In [ ]:
messages[messages['reactions_count'] > 0].sort_values(
    by='reactions_count', ascending=False)[['text', 'reactions_count']]

In [ ]:
messages.head(5)

In [12]:
train = messages[['text', 'author_username', 'author_id', 'reactions_count']]

In [13]:
def preproccess_text(text):
    # remove links
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^\w|\s]+', ' ', text)
    text = re.sub('\s+', ' ', text).strip()
    return text.lower()

train['text'] = train['text'].apply(preproccess_text)
# remove empty messages
train = train[train['text'].str.len() > 1]

In [14]:
def transform_to_input(train_messages: pd.DataFrame):
    # format "fake_name: message"
    unique_users = train_messages['author_username'].unique()
    fake = faker.Faker()
    fake_names = [fake.first_name() for i in range(len(unique_users))]
    fake_names_map = dict(zip(unique_users, fake_names))
    train_messages['author_fakename'] = train_messages['author_username'].map(fake_names_map)
    train_messages['combined'] = train_messages['author_fakename'] + ": " + train_messages['text']
    return '\n'.join(train_messages['combined'].values)


In [16]:
print(transform_to_input(train[15:0:-1]))

Olivia: if someone s ndcg output was and not please tell me what exactly was wrong
Taylor: wrong promtp in chatgpt
Jennifer: i had something like that i took prel in the formula not from the previous one i but from i
Michael: i think another task
Jennifer: ahh that s right
Jennifer: well then i thought idcg wrong
Jennifer: i just had all kinds of mistakes
Olivia: thank you i ll try it
Michael: quiz on ir tomorrow
Michael: i see thanks
Tamara: so or in the end
Robert: isn t it
Cathy: in moodle
Michelle: oops by chance it went forward to september
Christopher: who is retaking the axis is there any information on where the retake will take place
